In [10]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors import CellExecutionError
from nbclient.exceptions import CellControlSignal

## Get a list of notebooks to run

In [11]:
import sys
import os
sys.path.append(os.environ.get('NOTEBOOK_ROOT'))

# exclude 'machine_learning' because we do not have the data 
#   to test the Uruguay Random Forest notebooks.
# exclude 'UN_SDG_15_3_1.ipynb' due to no land change data (instructions in nbk).
# exclude 'ALOS_Land_Change.ipynb' due to no ALOS data.
# exclude 'Landslide_Identification_SLIP.iypnb' due to no TERRA ASTER data.
# exclude 'Forest_Change_VNSC.ipynb' due to no TERRA ASTER data.
# exclude 'Shallow_Water_Bathymetry.ipynb' due to no Landsat 8 Level 1 data.
# exclude 'ALOS_WASARD.ipynb' due to no ALOS data.
# exclude 'water_interoperability_similarity.ipynb' due to no Sentinel-2 data.
exclude_subpaths = ['.ipynb_checkpoints', 'experimental', 'IGARSS', 
                    'legacy', 'machine_learning', 'test', 'utils',  
                    'Automated_Testing.ipynb', 'UN_SDG_15_3_1.ipynb',
                    'ALOS_Land_Change.ipynb', 'Landslide_Identification_SLIP.ipynb',
                    'Forest_Change_VNSC.ipynb', 'Shallow_Water_Bathymetry.ipynb',
                    'ALOS_WASARD.ipynb', 'water_interoperability_similarity.ipynb',
                    ]
notebook_file_paths = []
for root, directories, files in os.walk('..', topdown=True):
    notebook_file_paths.extend([os.path.join(root, file) for file in files if file.endswith('.ipynb')])
notebook_file_paths = sorted(notebook_file_paths)

**Filter out notebooks using `exclude_subpaths`**

In [12]:
def is_filepath_excluded(filepath, exclude_subpaths):
    return any(list(map(lambda subpath: subpath in filepath, exclude_subpaths)))
notebook_file_paths = [filepath for filepath in notebook_file_paths if not is_filepath_excluded(filepath, exclude_subpaths)]

## Run the notebooks and record their status (e.g. working, error) to HTML as each completes

In [15]:
import re
import pandas
ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
full_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
error_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
success_report = pandas.DataFrame(columns=['Notebook', 'Status', 'Bug Description'])
for notebook_file_path in notebook_file_paths:
    print(f'Running {notebook_file_path}')
    run_result = {'Notebook': notebook_file_path, 'Status': 'Working', 'Bug Description': ''}
    with open(notebook_file_path, 'r+', encoding='utf-8') as notebook_file:
        notebook = nbformat.read(notebook_file, as_version=4)
        notebook_runner = ExecutePreprocessor(timeout=None)
        try:
            notebook_runner.preprocess(notebook, {'metadata': {'path': os.path.dirname(notebook_file_path)}})
        except CellExecutionError as err:
            run_result['Status'] = 'Error'
            run_result['Bug Description'] = err
        nbformat.write(notebook, notebook_file_path)
    full_report = full_report.append(run_result, ignore_index=True)
    if run_result['Status'] == 'Error':
        error_report = error_report.append(run_result, ignore_index=True)
    else:
        success_report = success_report.append(run_result, ignore_index=True)
    full_report.to_html('full_test_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    error_report.to_html('error_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})
    success_report.to_html('success_report.html', escape=False, formatters={'Bug Description': lambda e: ansi_escape.sub('', str(e).replace('\n', '<br/>'))})

Running ../DCAL/DCAL_Cloud_Statistics.ipynb
Running ../DCAL/DCAL_Custom_Mosaics.ipynb
Running ../DCAL/DCAL_Custom_Mosaics_Dask.ipynb
Running ../DCAL/DCAL_Land_Change.ipynb
Running ../DCAL/DCAL_Spectral_Products.ipynb
Running ../DCAL/DCAL_Vegetation_Phenology.ipynb
Running ../DCAL/DCAL_Water_Extents.ipynb
Running ../DCAL/DCAL_Water_WOFS.ipynb
Running ../DEM/SRTM_Product_Showcase_v2.ipynb
Running ../SAR/Sentinel1_Viewer.ipynb
Running ../UN_SDG/UN_SDG_11_3_1.ipynb
Running ../UN_SDG/UN_SDG_6_6_1.ipynb
Running ../animation/2D/GIF_Notebook.ipynb
Running ../animation/3D/GA_Water_3D_Reservoir/GA_Water_3DReservoir.ipynb
Running ../compositing/Composites.ipynb
Running ../compositing/Geomedians_and_Geomedoids.ipynb
Running ../feature_extraction/Clustering_Notebook.ipynb
Running ../general/Export.ipynb
Running ../general/Notebook_Template.ipynb
Running ../general/Shapefile_Masking.ipynb
Running ../scalability/Dask/Dask_Tutorial_Notebook.ipynb
Running ../training/ardc_training/Training_TaskA_Mosaic

### Export the results to a CSV

In [16]:
full_report['Bug Description'] = full_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
full_report.to_csv('full_test_report.csv')
error_report['Bug Description'] = error_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
error_report.to_csv('error_report.csv')
success_report['Bug Description'] = success_report['Bug Description'].map(lambda e: ansi_escape.sub('', str(e)))
success_report.to_csv('success_report_report.csv')

In [ ]:
report